In [1]:
%load_ext autoreload
import tensorflow as tf
import edward as ed
from edward.models import Normal, Categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
X_data, y_data = datasets.load_digits(return_X_y=True)
X_data = X_data/X_data.max()
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3)

In [ ]:
DIN = X_train.shape[1]
DOUT = len(set(y_train))
X = tf.placeholder(tf.float32, [None, DIN])
W = Normal(mu=tf.zeros([DIN, DOUT]), sigma=tf.ones([DIN, DOUT]))
b = Normal(mu=tf.zeros([DOUT]), sigma=tf.ones([DOUT]))
y = Categorical(logits=tf.matmul(X, W) + b)

qW = Normal(mu=tf.Variable(tf.random_normal([DIN, DOUT])),
            sigma=tf.nn.softplus(tf.Variable(tf.random_normal([DIN, DOUT]))))
qb = Normal(mu=tf.Variable(tf.random_normal([DOUT])),
            sigma=tf.nn.softplus(tf.Variable(tf.random_normal([DOUT]))))

In [ ]:
y.get_shape(), W.get_shape()

In [ ]:
inference = ed.KLqp({W: qW}, data={X: X_train, y: y_train}) 

In [ ]:
inference.run(n_samples=5, n_iter=10000)

In [ ]:
preds = []
for i in range(5):
    wv = qW.eval()
    bv = qb.eval()

    y_pred = (np.matmul(X_test, wv) + bv)
    preds.append(y_pred)

In [ ]:
preds = np.array(preds)

In [ ]:
(preds.mean(axis=0).argmax(axis=1) == y_test).std()

In [ ]:
y_pred.shape

In [22]:
%autoreload 2
import models

model = {
        'name': 'logistic_regression',
        'inference': 'variational',
        'n_iter': 1000,
        'n_samples': 5,
    }

model = models.load(model)

In [23]:
model.fit(X_train, y_train)

1000/1000 [100%] ██████████████████████████████ Elapsed: 13s | Loss: 711.135


In [28]:
(model.predict(X_test) == y_test).mean()

0.83333333333333337

In [40]:
pred = model.predict_proba(X_test)
sorted(zip(pred[0], range(10)))

[(-6.3548013470135629, 0),
 (-5.6645389255136251, 9),
 (-1.9681641133502126, 5),
 (-1.7925044894218445, 8),
 (-1.5101598249748349, 6),
 (1.708775972481817, 3),
 (1.717641226015985, 1),
 (1.9228819394484162, 2),
 (2.2161536570638418, 4),
 (7.9283710354939103, 7)]

In [43]:
pred[0]

array([-6.35480135,  1.71764123,  1.92288194,  1.70877597,  2.21615366,
       -1.96816411, -1.51015982,  7.92837104, -1.79250449, -5.66453893])